<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/Addestramento_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [5]:
import tensorflow as tf
import numpy as np

##Def variabili

In [1]:
#Scanner
scanner_name = 'DigitalPersona'

#salvataggio modello
model_save_name = 'VGG16'
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/Scanner/{scanner_name}/{model_save_name}" 

#load dataset training
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}'

##Parametri

In [2]:
batch_size=128
num_class=2

##Collegamento google drive

In [3]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Caricamento dataset pytorch (elimina)

In [106]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import shutil
import copy

data_transform=transforms.Compose([transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

train_dataset = datasets.ImageFolder(pathDataset,transform=data_transform,)

train_size = int(len(train_dataset) * 0.8)
val_size = len(train_dataset) - train_size

train_set, val_set = torch.utils.data.random_split(train_dataset, [train_size, val_size])

#splitter = DatasetValidationSplitter(len(trainset), 0.2)
#trainset_split = splitter.get_train_dataset(trainset)
#valset = splitter.get_val_dataset(trainset)

traingen = torch.utils.data.DataLoader(train_set, pin_memory=True, batch_size=batch_size, shuffle=True,num_workers=2)

###Riduzione dataset

In [ ]:
#riduzione train set
trainset.data=trainset.data[:100]

###Stampa dimensioni

In [ ]:
print("Dimensione trainset_split: "+str(len(trainset_split)))
print("Dimensione valset: "+str(len(valset)))
print("Dimensione testset: "+str(len(testset)))

###Nomi classi

In [ ]:
classes_name=trainset.classes
class_number=len(classes_name)
print(classes_name)
print(class_number)

##Caricamento dataset tensorflow (elimina)

In [92]:
import tensorflow as tf
import numpy as np
#tf.compat.v1.disable_eager_execution()

train_ds= tf.keras.utils.image_dataset_from_directory(
    pathDataset,
    validation_split=0.2,
    subset="training",
    seed=123,
    batch_size=batch_size,
    label_mode='categorical')

val_ds = tf.keras.utils.image_dataset_from_directory(
  pathDataset,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=batch_size,
  label_mode='categorical')

Found 2500 files belonging to 2 classes.
Using 2000 files for training.
Found 2500 files belonging to 2 classes.
Using 500 files for validation.


###Classi

In [93]:
class_names = train_ds.class_names
print(class_names)

['Live', 'Spoof']


###stampa immagini

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

###Stampa info dataset

In [94]:
print(len(train_ds)) #numero batch di dimensione 128

16


###Stampa info singolo elemento

In [122]:
train_iter=iter(train_ds)

In [123]:
image_batch, labels_batch = next(train_iter)

In [ ]:
print(image_batch)

In [97]:
print(image_batch[0].shape)
print(labels_batch[:5])

(256, 256, 3)
tf.Tensor(
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]], shape=(5, 2), dtype=float32)


In [98]:
im_shape=image_batch[0].shape

###Load model pre-trained

In [87]:
#creazione rete usando vgg16 preaddestrata e aggiungendo gli ultimi livelli per adattarla al problema di 10 classi
from tensorflow.keras import applications as applications

#importa rete vgg16 addestrata sul dataset imagenet, esclude gli ultimi livelli e come input pongo dimensioni 96,96,3 
model_conv = applications.VGG16(weights='imagenet', include_top=False, input_shape=(im_shape[0],im_shape[1],im_shape[2]))

# Freeze all the layers (non modifico i pesi dei livelli inferiori)
for layer in model_conv.layers[:]:
    layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


###training
> per riaddestrare eseguire solo questa sezione



####Aggiunta classificatore

In [100]:
from tensorflow.keras import layers,models,losses,optimizers,applications
import numpy as np

# creo un modello aggiungendo livelli alla rete importata
model = models.Sequential()
# Add the vgg convolutional base model
model.add(model_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax')) #10: numero di valori in uscita (le classi)

#stampa info del modello
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              33555456  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2050      
Total params: 48,272,194
Trainable params: 33,557,506
Non-trainable params: 14,714,688
_________________________________________________________________


####Configura parametri modello

In [101]:
# Configure the model for training (setta i parametri del modello)
model.compile(
        loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"]
    )

####Addestramento

In [ ]:
history= model.fit(train_ds,epochs=5,validation_data=val_ds)

##Caricamento dataset tensorflow 2 (def)

caricamento dataset

In [6]:
train_ds= tf.keras.utils.image_dataset_from_directory(
    pathDataset,
    label_mode='categorical')


Found 2000 files belonging to 2 classes.


calcolo dimensioni dataset

In [ ]:
#cambia: usa conteggio file nella cartella per velocizzare
dim_set=0
for image_batch, labels_batch in train_ds:
  dim_set=dim_set+image_batch.get_shape()[0]
print(dim_set)  

calcolo dimensioni singola immagine

In [10]:
image_batch, labels_batch = next(iter(train_ds))
im_shape=image_batch[0].shape

In [11]:
x_train = np.zeros((dim_set, im_shape[0], im_shape[1], im_shape[2]), dtype=np.uint8)
y_train = np.zeros((dim_set,num_class), dtype=np.uint8)

In [12]:
x_train.shape

(2000, 256, 256, 3)

In [14]:
i=0
j=0
for image_batch, labels_batch in train_ds:
  j=i+image_batch.get_shape()[0]
  #print("i: "+str(i))
  #print("j: "+str(j))
  x_train[i : j,:,:,:]=image_batch
  y_train[i : j ,:]=labels_batch
  i=j

del i,j  

preprocessing immagini

In [16]:
min_, max_ = np.amin(x_train), np.amax(x_train)
x_train = (x_train - min_) / (max_ - min_)

###Load model pre-trained

In [17]:
#creazione rete usando vgg16 preaddestrata e aggiungendo gli ultimi livelli per adattarla al problema di 10 classi
from tensorflow.keras import applications as applications

#importa rete vgg16 addestrata sul dataset imagenet, esclude gli ultimi livelli e come input pongo dimensioni 96,96,3 
model_conv = applications.VGG16(weights='imagenet', include_top=False, input_shape=(im_shape[0],im_shape[1],im_shape[2]))

# Freeze all the layers (non modifico i pesi dei livelli inferiori)
for layer in model_conv.layers[:]:
    layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


###training
> per riaddestrare eseguire solo questa sezione



####Aggiunta classificatore

In [18]:
from tensorflow.keras import layers,models,losses,optimizers,applications
import numpy as np

# creo un modello aggiungendo livelli alla rete importata
model = models.Sequential()
# Add the vgg convolutional base model
model.add(model_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_class, activation='softmax')) #10: numero di valori in uscita (le classi)

#stampa info del modello
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              33555456  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 48,272,194
Trainable params: 33,557,506
Non-trainable params: 14,714,688
_________________________________________________________________


####Configura parametri modello

In [19]:
# Configure the model for training (setta i parametri del modello)
model.compile(
        loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"]
    )

####Addestramento

In [20]:
history= model.fit(x_train,y_train,epochs=2,batch_size=128,validation_split=0.2)

Epoch 1/2
13/13 [==============================] - 103s 3s/step - loss: 1.0203 - accuracy: 0.6800 - val_loss: 0.3003 - val_accuracy: 0.8750
Epoch 2/2
13/13 [==============================] - 18s 1s/step - loss: 0.2667 - accuracy: 0.8856 - val_loss: 0.1748 - val_accuracy: 0.9300


#**Salvataggio modello**

In [ ]:
#salva modello su drive
model.save(path_model_save)